In [3]:
%%capture --no-stderr
%pip install -U langchain_openai langgraph python-dotenv

In [35]:
import pandas as pd
import multi_agent_model
from multi_agent_model import analyze_reflection, app, Analyst

In [51]:
test_data = '/content/drive/MyDrive/BFH/rflect/data/data_v3/500_v3_cleaned_with_topic.xlsx'
test_df = pd.read_excel(test_data)

In [52]:
evaluation_50 = test_df[0:50]
# evaluation_50[['Topic', 'question', 'reflection_text', 'course_name']]

## Single Example Evaluation

In [53]:
# Example data
topic = "Data Science and Technology"
reflection_input = "The difference between Regressions and Classifications. For me these topics were not completely new but far away for applying them  after the last module the fog lifted and I have a clearer view again even though there is still to learn on application usage and limitations of these subjects"
course_name = "CAS Data Science and Machine Learning"
question = "What do you know now that you didnt know before the last module"

# Call the analyze_reflection function
result = analyze_reflection(topic, reflection_input, course_name, question)

# Print the result
print(result)

{'grades': {'Description': 1, 'Feelings': 0, 'Evaluation': 1, 'Analysis': 1, 'Conclusion': 1, 'Action Plan': 0}, 'feedback': "You've done well in articulating your learning journey and emotional growth. To deepen your reflection, consider sharing specific feelings you experienced during the learning process. Additionally, for your action plan, what concrete steps will you take to further explore regressions and classifications? Perhaps set a goal to complete a project or seek additional resources. How will you measure your progress in the coming months?"}


## Dataset Evaluation

Evaluation on the 50 human-labeled dataset

### data pre-process

In [54]:

def check_null_values(df, columns):
    for column in columns:
        if df[column].isnull().any():
            print(f"Column '{column}' contains null values.")
            return True  # Return True if nulls are found
    return False  # Return False if no nulls are found

columns_to_check = ['Topic', 'question', 'reflection_text', 'course_name']

# Check for null values
if check_null_values(test_df, columns_to_check):
    print("Null values were found in the specified columns.")
else:
    print("No null values were found in the specified columns.")


def process_columns(df, columns):
    for column in columns:
        df.loc[:, column] = df[column].fillna('').astype(str)  # Using .loc for explicit assignment
    return df


Column 'reflection_text' contains null values.
Null values were found in the specified columns.


In [62]:
columns_to_process = ['Topic', 'question', 'reflection_text', 'course_name']
evaluation_50 = process_columns(evaluation_50, columns_to_process)

In [63]:
evaluation_50['results'] = evaluation_50.apply(lambda row:
                                               analyze_reflection(row['Topic'], row['reflection_text'], row['course_name'], row['question']), axis=1)

<ipython-input-63-6e24ca2387b8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  evaluation_50['results'] = evaluation_50.apply(lambda row:


## parse the gpt-4o-mini results

In [67]:

# Create a new dataframe to store parsed results
results_df = pd.DataFrame(columns=['Description', 'Feeling', 'Evaluation', 'Analysis', 'Conclusion', 'Action Plan', 'feedback'])

# Parse the results and populate the new dataframe
for index, row in evaluation_50.iterrows():
    result = row['results']
    grades = result.get('grades', {})  # Handle missing grades gracefully
    new_row = {
        'Description': grades.get('Description'),
        'Feeling': grades.get('Feelings'),
        'Evaluation': grades.get('Evaluation'),
        'Analysis': grades.get('Analysis'),
        'Conclusion': grades.get('Conclusion'),
        'Action Plan': grades.get('Action Plan'),
        'feedback': result.get('feedback')
    }
    results_df = pd.concat([results_df, pd.DataFrame([new_row])], ignore_index=True)


In [69]:
evaluated_50_path = '/content/drive/MyDrive/BFH/rflect/data/data_v3/50_evaluation.xlsx'
results_df.to_excel(evaluated_50_path, index=False)